In [ ]:
from PIL import Image, ImageDraw
from IPython.display import display
from playsound import playsound
import face_recognition
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display
import cv2
import glob
import sys
import time
import ctypes
from tqdm import tqdm

threshold = 0.6

def lock_workstation():
    ctypes.windll.user32.LockWorkStation()
    
def take_camera_shot(source=0):
    cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 
    ret,frame = cap.read() # return a single frame in variable `frame`
    cv2.destroyAllWindows()
    cap.release()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame

def load_faces_from_folder(path):
    image_list = []
    names_list = []
    for filename in glob.glob(path + '/*.jpg'):
        name = filename.split('\\')[-1].split('.')[0]
        im=face_recognition.load_image_file(filename)
        image_list.append(im)
        names_list.append(name)
    return image_list, names_list;

def encode_known_faces(known_faces_images):
    known_faces_encodings = []
    for img in known_faces_images:
        face_encoding = face_recognition.face_encodings(img)[0]
        known_faces_encodings.append(face_encoding)
    return known_faces_encodings

def initialize(path):
    known_faces_images, known_faces_names = load_faces_from_folder(path)
    known_faces_encodings = encode_known_faces(known_faces_images)
    print('Learned encoding for', len(known_faces_encodings), 'images.')
    
    return known_faces_encodings, known_faces_names;

def convert_to_cv2_image(pil_image):
    open_cv_image = np.array(pil_image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    return cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2RGB)

def authorize(unknown_image, known_face_encodings, known_face_names, counter = 0, secure_mode = False):
    #unknown_image = face_recognition.load_image_file("3.jpg")

    all_names = []
    # Find all the faces and face encodings in the unknown image
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    # Convert the image to a PIL-format image so that we can draw on top of it with the Pillow library
    # See http://pillow.readthedocs.io/ for more about PIL/Pillow
    pil_image = Image.fromarray(unknown_image)
    # Create a Pillow ImageDraw Draw instance to draw with
    draw = ImageDraw.Draw(pil_image)

    # Loop through each face found in the unknown image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=threshold)

        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index].split("_")[0]
            all_names.append(name)
        
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


    # Remove the drawing library from memory as per the Pillow docs
    del draw

    if secure_mode:
        if len(all_names) > 0:
            print(all_names)
        if len(all_names) > 0 and 'Pawel' not in all_names:
            lock_workstation()
            #playsound('HeadshotSoundEffect.mp3')
    
    # Display the resulting image
    #display(pil_image)
    pil_image.save("predict" + str(counter) + ".jpg")
    
    return convert_to_cv2_image(pil_image)


In [ ]:
encodings, names = initialize("known_people")

In [ ]:
def authorize_from_folder(path):
    counter = 1
    unknown_images, random = load_faces_from_folder(path)
    for img in unknown_images:
        authorize(img, encodings, names, counter)
        counter = counter + 1

In [ ]:
def video_auth(encodings, names):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out1 = cv2.VideoWriter('output.avi', fourcc, 20.0, (640,480))
    cap1 = cv2.VideoCapture(0) 
    for i in tqdm(range(0, 100)):
        ret,frame = cap1.read()
        #cv2.imshow('frame', frame)
        #image = authorize(frame, encodings, names)
        out1.write(frame)
        #cv2.imshow('authorization', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap1.release()
    out1.release()
    out2 = cv2.VideoWriter('output_auth.avi', fourcc, 20.0, (640,480))
    cap2 = cv2.VideoCapture('output.avi')         
    for i in tqdm(range(0,100)):
        ret,frame = cap2.read()
        #cv2.imshow('frame', frame)
        image = authorize(frame, encodings, names)
        out2.write(image)
        #cv2.imshow('authorization', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap2.release()
    out2.release()
    cv2.destroyAllWindows()  
    

In [5]:
video_auth(encodings, names)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:24<00:00,  1.02s/it]


In [ ]:
# live recording with sucure mode

def realtime_detection():
    cap1 = cv2.VideoCapture(0) 
    start = time.time()
    for i in tqdm(range(0, 100)):
        ret,frame = cap1.read()
        image = authorize(frame, encodings, names, secure_mode = True)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap1.release()
    
    print('Detection time: ', time.time() - start)

In [ ]:
# live recording with showing prediction output

def realtime_detection2():
    cap1 = cv2.VideoCapture(0) 
    start = time.time()
    for i in tqdm(range(0, 100)):
        ret,frame = cap1.read()
        image = authorize(frame, encodings, names, secure_mode = False)
        cv2.imshow("dsaf", image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap1.release()
    
    print('Detection time: ', time.time() - start)

In [ ]:
realtime_detection()

In [13]:
def batch_find_locations():
    

    # This code finds all faces in a list of images using the CNN model.
    #
    # This demo is for the _special case_ when you need to find faces in LOTS of images very quickly and all the images
    # are the exact same size. This is common in video processing applications where you have lots of video frames
    # to process.
    #
    # If you are processing a lot of images and using a GPU with CUDA, batch processing can be ~3x faster then processing
    # single images at a time. But if you aren't using a GPU, then batch processing isn't going to be very helpful.
    #
    # PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read the video file.
    # OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
    # specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

    # Open video file
    video_capture = cv2.VideoCapture("output.avi")

    frames = []
    frame_count = 0

    while video_capture.isOpened():
        # Grab a single frame of video
        ret, frame = video_capture.read()

        print(frame)
        # Bail out when the video file ends
        if not ret:
            break

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        frame = frame[:, :, ::-1]

        # Save each frame of the video to a list
        frame_count += 1
        frames.append(frame)

        # Every 128 frames (the default batch size), batch process the list of frames to find faces
        if len(frames) == 128:
            batch_of_face_locations = face_recognition.batch_face_locations(frames, number_of_times_to_upsample=0)

            # Now let's list all the faces we found in all 128 frames
            for frame_number_in_batch, face_locations in enumerate(batch_of_face_locations):
                number_of_faces_in_frame = len(face_locations)

                frame_number = frame_count - 128 + frame_number_in_batch
                print("I found {} face(s) in frame #{}.".format(number_of_faces_in_frame, frame_number))

                for face_location in face_locations:
                    # Print the location of each face in this frame
                    top, right, bottom, left = face_location
                    print(" - A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

            # Clear the frames array to start the next batch
            frames = []

In [14]:
batch_find_locations()

[[[135 135 126]
  [135 135 126]
  [135 135 126]
  ...
  [130 130 128]
  [130 130 128]
  [130 130 128]]

 [[135 135 126]
  [135 135 126]
  [135 135 126]
  ...
  [130 130 128]
  [130 130 128]
  [130 130 128]]

 [[135 135 126]
  [135 135 126]
  [135 135 126]
  ...
  [130 130 128]
  [130 130 128]
  [130 130 128]]

 ...

 [[ 37  44  22]
  [ 36  43  21]
  [ 34  38  17]
  ...
  [ 50  39  30]
  [ 50  39  30]
  [ 50  39  30]]

 [[ 34  44  22]
  [ 31  41  19]
  [ 28  38  16]
  ...
  [ 50  39  30]
  [ 50  39  30]
  [ 50  39  30]]

 [[ 33  43  21]
  [ 31  41  19]
  [ 28  38  16]
  ...
  [ 50  39  30]
  [ 50  39  30]
  [ 50  39  30]]]
[[[137 135 131]
  [137 135 131]
  [137 135 131]
  ...
  [130 130 128]
  [130 130 128]
  [130 130 128]]

 [[137 135 131]
  [137 135 131]
  [137 135 131]
  ...
  [130 130 128]
  [130 130 128]
  [130 130 128]]

 [[137 135 131]
  [137 135 131]
  [137 135 131]
  ...
  [130 130 128]
  [130 130 128]
  [130 130 128]]

 ...

 [[ 30  38  21]
  [ 28  36  19]
  [ 27  35  18]
  ..

  [ 67  47  39]]]
[[[164 162 153]
  [164 162 153]
  [164 162 153]
  ...
  [157 157 155]
  [157 157 155]
  [157 157 155]]

 [[164 162 153]
  [164 162 153]
  [164 162 153]
  ...
  [157 157 155]
  [157 157 155]
  [157 157 155]]

 [[164 162 153]
  [164 162 153]
  [164 162 153]
  ...
  [157 157 155]
  [157 157 155]
  [157 157 155]]

 ...

 [[ 38  41  31]
  [ 37  40  30]
  [ 37  41  29]
  ...
  [ 67  47  39]
  [ 67  47  39]
  [ 67  47  39]]

 [[ 38  41  31]
  [ 37  40  30]
  [ 37  41  29]
  ...
  [ 66  46  38]
  [ 66  46  38]
  [ 66  46  38]]

 [[ 38  41  31]
  [ 37  40  30]
  [ 37  41  29]
  ...
  [ 66  46  38]
  [ 66  46  38]
  [ 66  46  38]]]
[[[161 160 154]
  [161 160 154]
  [161 160 154]
  ...
  [157 157 155]
  [157 157 155]
  [157 157 155]]

 [[161 160 154]
  [161 160 154]
  [161 160 154]
  ...
  [157 157 155]
  [157 157 155]
  [157 157 155]]

 [[161 160 154]
  [161 160 154]
  [161 160 154]
  ...
  [157 157 155]
  [157 157 155]
  [157 157 155]]

 ...

 [[ 38  41  31]
  [ 37  40  30]
  

  [ 84  59  50]]]
[[[197 190 182]
  [197 190 182]
  [197 190 182]
  ...
  [184 181 180]
  [184 181 180]
  [184 181 180]]

 [[197 190 182]
  [197 190 182]
  [197 190 182]
  ...
  [184 181 180]
  [184 181 180]
  [184 181 180]]

 [[197 190 182]
  [197 190 182]
  [197 190 182]
  ...
  [184 181 180]
  [184 181 180]
  [184 181 180]]

 ...

 [[ 74  63  47]
  [ 69  58  42]
  [ 66  55  39]
  ...
  [ 86  61  52]
  [ 86  61  52]
  [ 86  61  52]]

 [[ 65  56  40]
  [ 59  50  34]
  [ 59  50  34]
  ...
  [ 85  60  51]
  [ 85  60  51]
  [ 85  60  51]]

 [[ 67  58  42]
  [ 62  53  37]
  [ 59  50  34]
  ...
  [ 84  59  50]
  [ 84  59  50]
  [ 84  59  50]]]
[[[197 191 181]
  [197 191 181]
  [197 191 181]
  ...
  [182 182 180]
  [179 182 180]
  [179 182 180]]

 [[197 191 181]
  [197 191 181]
  [197 191 181]
  ...
  [182 182 180]
  [179 182 180]
  [179 182 180]]

 [[197 191 181]
  [197 191 181]
  [197 191 181]
  ...
  [182 182 180]
  [179 182 180]
  [179 182 180]]

 ...

 [[ 75  64  48]
  [ 71  60  44]
  

  [ 93  68  52]]]
[[[214 207 199]
  [214 207 199]
  [214 207 199]
  ...
  [204 199 198]
  [204 199 198]
  [204 199 198]]

 [[214 207 199]
  [214 207 199]
  [214 207 199]
  ...
  [204 199 198]
  [204 199 198]
  [204 199 198]]

 [[214 207 199]
  [214 207 199]
  [214 207 199]
  ...
  [203 198 197]
  [204 199 198]
  [204 199 198]]

 ...

 [[ 91  76  58]
  [ 87  72  54]
  [ 78  65  47]
  ...
  [ 94  67  56]
  [ 95  68  57]
  [ 95  68  57]]

 [[ 76  68  49]
  [ 72  64  45]
  [ 67  62  42]
  ...
  [ 94  67  56]
  [ 94  67  56]
  [ 94  67  56]]

 [[ 76  68  49]
  [ 72  64  45]
  [ 65  60  40]
  ...
  [ 94  67  56]
  [ 94  67  56]
  [ 94  67  56]]]
[[[214 208 198]
  [214 208 198]
  [214 208 198]
  ...
  [206 198 198]
  [206 198 198]
  [206 198 198]]

 [[214 208 198]
  [214 208 198]
  [214 208 198]
  ...
  [205 197 197]
  [206 198 198]
  [206 198 198]]

 [[214 208 198]
  [214 208 198]
  [214 208 198]
  ...
  [205 197 197]
  [205 197 197]
  [205 197 197]]

 ...

 [[ 93  78  60]
  [ 89  74  56]
  

  [102  71  72]]]
[[[230 222 210]
  [230 222 210]
  [230 222 210]
  ...
  [222 212 212]
  [223 213 213]
  [223 213 213]]

 [[230 222 210]
  [230 222 210]
  [230 222 210]
  ...
  [222 212 212]
  [222 212 212]
  [222 212 212]]

 [[230 222 210]
  [230 222 210]
  [230 222 210]
  ...
  [221 211 211]
  [221 211 211]
  [221 211 211]]

 ...

 [[109  84  68]
  [101  76  60]
  [ 90  68  51]
  ...
  [102  73  69]
  [102  73  69]
  [102  73  69]]

 [[102  74  61]
  [ 96  68  55]
  [ 86  63  49]
  ...
  [102  73  69]
  [102  73  69]
  [102  73  69]]

 [[101  73  60]
  [ 93  65  52]
  [ 83  60  46]
  ...
  [102  73  69]
  [102  73  69]
  [102  73  69]]]
[[[230 222 210]
  [230 222 210]
  [230 222 210]
  ...
  [221 211 211]
  [222 212 212]
  [222 212 212]]

 [[230 222 210]
  [230 222 210]
  [230 222 210]
  ...
  [221 211 211]
  [221 211 211]
  [221 211 211]]

 [[230 222 210]
  [230 222 210]
  [230 222 210]
  ...
  [219 211 211]
  [219 211 211]
  [219 211 211]]

 ...

 [[109  84  68]
  [101  76  60]
  

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out2 = cv2.VideoWriter('output_auth.avi', fourcc, 20.0, (640,480))
cap2 = cv2.VideoCapture('output.avi')         
for i in tqdm(range(0,100)):
    ret,frame = cap2.read()
    #cv2.imshow('frame', frame)
    image = authorize(frame, encodings, names)
    out2.write(image)
    #cv2.imshow('authorization', image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap2.release()
out2.release()
cv2.destroyAllWindows()  

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:53<00:00,  1.38s/it]
